<a href="https://www.kaggle.com/code/gpreda/prompt-gemma-like-a-pro?scriptVersionId=170800187" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<center><img src="https://res.infoq.com/news/2024/02/google-gemma-open-model/en/headerimage/generatedHeaderImage-1708977571481.jpg" width="400"></center>

# Introduction 

This Notebook is just for me to check a bit Gemma model. 
Will just load Gemma and demonstrate some very simple use cases.
I intend to learn from this experience so that I can then build someting a bit more complex.

# Install Keras NLP and Keras

In [1]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.


# Import packages

In [2]:
import keras
import keras_nlp
import os
from time import time

# Select the desired backend for Keras. Options: "jax", "tensorflow", or "torch".
os.environ["KERAS_BACKEND"] = "jax"  # Adjust as needed.

# Specific to the JAX backend, this setting helps avoid memory fragmentation, ensuring more efficient resource use.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

2024-04-07 12:24:26.435785: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 12:24:26.435884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 12:24:26.564911: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load Gemma Causal LLM through Keras NLP 

In [3]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


# Check Gemma parameters

In [4]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

# Define and run a simple prompt (geography question)

In [5]:
prompt = """
You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.
Question: {question}
Answer:
"""
response = gemma_lm.generate(prompt.format(question="What is the surface temperature of the Moon?"), max_length=256)
print(response)

I0000 00:00:1712492764.464763      26 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1712492764.535654      26 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1712492764.683584      26 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.
Question: What is the surface temperature of the Moon?
Answer:
The surface temperature of the Moon is extremely cold, with an average temperature of -173°C (-283°F).


# Format the output 

In [6]:
from IPython.display import display, Markdown

def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [7]:
t = time()
response = gemma_lm.generate(prompt.format(question="What is the surface temperature of the Moon?"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.


**<font color='red'>Question:</font>** What is the surface temperature of the Moon?


**<font color='green'>Answer:</font>**
The surface temperature of the Moon is extremely cold, with an average temperature of -173°C (-283°F).



**<font color='magenta'>Total time:</font>** 0.81 sec.

# More such questions

Make sure he is really confused. He doesn't know European, ancient World or Japanese history.

In [8]:
t = time()
response = gemma_lm.generate(prompt.format(question="When was the 30 years war?"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.


**<font color='red'>Question:</font>** When was the 30 years war?


**<font color='green'>Answer:</font>**
The context does not provide any information about the 30 years war, so I cannot answer this question from the provided context.



**<font color='magenta'>Total time:</font>** 0.76 sec.

In [9]:
t= time()
response = gemma_lm.generate(prompt.format(question="What is a graphe paranomon?"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.


**<font color='red'>Question:</font>** What is a graphe paranomon?


**<font color='green'>Answer:</font>**
A graphe paranomon is a type of graph that is used to model a relationship between two sets of objects.



**<font color='magenta'>Total time:</font>** 0.69 sec.

In [10]:
t = time()
response = gemma_lm.generate(prompt.format(question="Who was the next shogon after Yeiatsu Tokugawa?"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.


**<font color='red'>Question:</font>** Who was the next shogon after Yeiatsu Tokugawa?


**<font color='green'>Answer:</font>**
I am unable to provide specific information about the next shogun, as I do not have access to real-time or comprehensive data.



**<font color='magenta'>Total time:</font>** 0.76 sec.

# Try another one (math problem)

In [11]:
prompt = """
You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.
Question: {question}
Answer:
"""
t = time()
response = gemma_lm.generate(prompt.format(question="Please write a function in Python to calculate the area of a circle of radius r"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.


**<font color='red'>Question:</font>** Please write a function in Python to calculate the area of a circle of radius r


**<font color='green'>Answer:</font>**
```python
def area_of_circle(radius):
  """
  Calculates the area of a circle of radius r.

  Args:
    radius: The radius of the circle in meters.

  Returns:
    The area of the circle in square meters.
  """

  # Calculate the area of the circle.
  area = math.pi * radius ** 2

  return area
```



**<font color='magenta'>Total time:</font>** 2.22 sec.

## Let's test more Python

In [12]:
t = time()
response = gemma_lm.generate(prompt.format(question="Please write a function in Python to order a list"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.


**<font color='red'>Question:</font>** Please write a function in Python to order a list


**<font color='green'>Answer:</font>**
```python
def order_list(list1):
  """
  Orders a list in ascending order.

  Args:
    list1: The list to order.

  Returns:
    The ordered list.
  """

  # Sort the list in ascending order.
  list1.sort()

  return list1


# Example usage.
list1 = [5, 2, 8, 1, 9]
ordered_list = order_list(list1)
print(ordered_list)
```



**<font color='magenta'>Total time:</font>** 2.84 sec.

## Some more Python

In [13]:
t = time()
response = gemma_lm.generate(prompt.format(
            question="""Please write a class in Python 
                        to model a phone book (storing name, surname, address, phone) 
                        with add, delete, order by name, search operations.
                        The class should store a list of contacts, each
                        with name, surname, address, phone information stored.
                        """), 
            max_length=1024)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))

W0000 00:00:1712492795.108114      26 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.


**<font color='red'>Question:</font>** Please write a class in Python 
                        to model a phone book (storing name, surname, address, phone) 
                        with add, delete, order by name, search operations.
                        The class should store a list of contacts, each
                        with name, surname, address, phone information stored.
                        


**<font color='green'>Answer:</font>**
```python
class PhoneBook:

    def __init__(self, contacts):
        self.contacts = contacts

    def add(self, name, surname, address, phone):
        self.contacts.append({"name": name, "surname": surname, "address": address, "phone": phone})

    def delete(self, name):
        for contact in self.contacts:
            if contact["name"] == name:
                self.contacts.remove(contact)
                return True

    def order_by_name(self):
        self.contacts.sort(key=lambda x: x["name"])

    def search(self, name):
        for contact in self.contacts:
            if contact["name"] == name:
                return contact

# Create a phone book with some contacts
phone_book = PhoneBook([
    {"name": "John Smith", "surname": "Doe", "address": "123 Main Street", "phone": "555-123-4567"},
    {"name": "Jane Doe", "surname": "Smith", "address": "456 Elm Street", "phone": "555-234-5678"},
    {"name": "Peter Parker", "surname": "Parker", "address": "789 Oak Street", "phone": "555-345-6789"},
])

# Add some contacts to the phone book
phone_book.add("Mary Johnson", "Williams", "101 Main Street", "555-567-8901")
phone_book.add("David Lee", "Anderson", "202 Elm Street", "555-678-9012")

# Delete a contact from the phone book
phone_book.delete("John Smith")

# Order the contacts by name
phone_book.order_by_name()

# Search for a contact by name
print(phone_book.search("Jane Doe"))
```



**<font color='magenta'>Total time:</font>** 32.04 sec.

# Multiple parameters questions

I create here a prompt with multiple parameters and I ask a variety of questions.

## Best food in France

In [14]:
prompt = """
You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.
Question: What are the {adjective} {number} {items} from {place}?
Answer:
"""
t = time()
response = gemma_lm.generate(prompt.format(
    adjective="best",
    number="3",
    items="food",
    place="France"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.


**<font color='red'>Question:</font>** What are the best 3 food from France?


**<font color='green'>Answer:</font>**
- Croque monsieur
- Steak frites
- Crème brûlée

## Best attractions in Italy

In [15]:
t = time()
response = gemma_lm.generate(prompt.format(
    adjective="best",
    number="five",
    items="attractions",
    place="Italy"
    ), 
    max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))                


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.


**<font color='red'>Question:</font>** What are the best five attractions from Italy?


**<font color='green'>Answer:</font>**
- The Colosseum
- The Vatican City
- The Trevi Fountain
- The Duomo
- The Uffizi Gallery



**<font color='magenta'>Total time:</font>** 0.74 sec.

## Most affordable places to retire in Spain

In [16]:
t = time()
response = gemma_lm.generate(prompt.format(
    adjective="most affordable",
    number="two",
    items="places to retire",
    place="Spain"
    ), 
    max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.


**<font color='red'>Question:</font>** What are the most affordable two places to retire from Spain?


**<font color='green'>Answer:</font>**
- **Seville**
- **Granada**

Explanation:

- Seville is a city in southern Spain with a lower cost of living compared to other major cities in the country.
- Granada is a city in the south of Spain with a lower cost of living compared to other major cities in the country.



**<font color='magenta'>Total time:</font>** 1.61 sec.

## Less known but great places to stay in Romania

In [17]:
t = time()
response = gemma_lm.generate(prompt.format(
    adjective="Less known but great",
    number="4",
    items="places to stay",
    place="Romania"
    ), 
    max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.


**<font color='red'>Question:</font>** What are the Less known but great 4 places to stay from Romania?


**<font color='green'>Answer:</font>**
- The Black Sea Coast
- The Danube Delta
- The Carpathian Mountains
- The Bucovina Region



**<font color='magenta'>Total time:</font>** 0.72 sec.

## Best commedies by Shakespeare

In [18]:
t = time()
response = gemma_lm.generate(prompt.format(
    adjective="best",
    number="3",
    items="commedies",
    place="Shakespeare entire creation"
    ), 
    max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.


**<font color='red'>Question:</font>** What are the best 3 commedies from Shakespeare entire creation?


**<font color='green'>Answer:</font>**
- As You Like It
- A Midsummer Night's Dream
- Macbeth



**<font color='magenta'>Total time:</font>** 0.54 sec.

## Most important battles from WW2

In [19]:
t = time()
response = gemma_lm.generate(prompt.format(
    adjective="most important",
    number="5",
    items="battles",
    place="WW2"
    ), 
    max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.


**<font color='red'>Question:</font>** What are the most important 5 battles from WW2?


**<font color='green'>Answer:</font>**
- Battle of Stalingrad
- Battle of Kursk
- Battle of Berlin
- Battle of D-Day
- Battle of Moscow



**<font color='magenta'>Total time:</font>** 0.84 sec.

# Reasoning like Einstein will do 

In [20]:
prompt = """
You are a math professor, smart but cool.
Background: A train traveling from Bucharest to Ploiesti (60 km distance) has the speed of 60 km/h. 
The train starts in Bucharest and travels until Ploiesti, once, only in this direction.
A swallow, flying with 90 km/h, fly from Ploiesti to the moving train.
When it reaches the train, the swallow flies back toward Ploiesti,
ahead of the train. At Ploiesti turns again back and continues to fly back and forth 
(between the train approaching Ploiesti and Ploiesti) until the train reaches Ploiesti. 
The swallow will fly continously all the time the train is traveling from Bucharest to Ploiesti.
Reasoning: Think step by step. Explain your reasoning.
Question: {question}
Answer:
"""
t = time()
response = gemma_lm.generate(prompt.format(question="How many kilometers will travel totally the swallow?"), 
    max_length=512)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))

W0000 00:00:1712492831.436113      26 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



You are a math professor, smart but cool.
Background: A train traveling from Bucharest to Ploiesti (60 km distance) has the speed of 60 km/h. 
The train starts in Bucharest and travels until Ploiesti, once, only in this direction.
A swallow, flying with 90 km/h, fly from Ploiesti to the moving train.
When it reaches the train, the swallow flies back toward Ploiesti,
ahead of the train. At Ploiesti turns again back and continues to fly back and forth 
(between the train approaching Ploiesti and Ploiesti) until the train reaches Ploiesti. 
The swallow will fly continously all the time the train is traveling from Bucharest to Ploiesti.


**<font color='blue'>Reasoning:</font>** Think step by step. Explain your reasoning.


**<font color='red'>Question:</font>** How many kilometers will travel totally the swallow?


**<font color='green'>Answer:</font>**
The swallow will travel a total distance of 120 km. 

**Explanation:**

1. The swallow flies from Ploiesti to the train at 90 km/h. Since the train travels at 60 km/h, the swallow will be flying relative to the train for a total of 60 minutes (1 hour) while traveling from Ploiesti to the train. 
2. After 60 minutes, the swallow turns back and flies back to Ploiesti. 
3. When the train reaches Ploiesti again, the swallow flies back to the train at 90 km/h. 
4. Since the train is already moving, the swallow will continue to fly back and forth between the train and Ploiesti as long as the train is traveling. 
5. The total distance the swallow will travel is the sum of the distances it travels from Ploiesti to the train and from the train to Ploiesti. 
6. Therefore, the total distance the swallow will travel is 60 minutes x 90 km/h + 60 minutes x 90 km/h = 120 km.



**<font color='magenta'>Total time:</font>** 25.79 sec.

# Conclusions


## Using gemma_2b_en

Initially, I used the model `gemma_2b_en` and I was not really happy with the results. The main problems I found were:
* The model will answer incorectly to the question
* The model will answer to more question than the one answered
* The model will stop suddently in the middle of a phrase

## Using gemma_instruct_2b_en

Then, I got a good advice in the comments from one of the Kagglers (thank you @paultimothymooney) and switched to `gemma_instruct_2b_en` with much better results.

Here is the analysis of results with `gemma_instruct_2b_en`:
* General question: answer to the point, and the answer seems sensible. Well, it is actually not entirely correct, but it is close. And the question did not clarify if it is during solar exposure or in the shadow. 
* It is clear that the model doesn't know history.
* Math problem, with answer as Python code: correct, and code is documented.
* More Python code: if too complex, start to make mistakes.
* Multi-parameters question: all answers are correct. 
* More difficult math problem, requiring reasoning: the answer is not correct, because the reasoning is not correct but, to be fair, ... the problem contains a paradox. Did you (intelligent human reading this) notice what is the paradox?


We were also evaluating the time for the response of various queries, with different `max_length` parameter. From less than one sec., for `max_length` = 256, it can raise to 1/2 min or more, for `max_length` 512 or 1024.